In [15]:
import os
import random
import lxml
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from typing import List, Dict, Tuple, Union
import string
# nltk.download('punkt')
# nltk.download('stopwords')

ORIGINAL_PATH = os.path.join(os.getcwd(), os.pardir, os.pardir, 'Dataset', 'CSE508_Winter2023_Dataset', 'CSE508_Winter2023_Dataset')
ALTERED_PATH = os.path.join(os.getcwd(), 'DatasetAlter')


In [16]:
ORIGINAL_PATH

'D:\\Projects\\IR\\TestA3\\Code\\Q2\\..\\..\\Dataset\\CSE508_Winter2023_Dataset\\CSE508_Winter2023_Dataset'

In [17]:
def create_alter(seed=1):

	random.seed(seed)
	random_samples = random.sample(range(1, 1401), 5)

	if not os.path.exists(ALTERED_PATH):
		os.makedirs(ALTERED_PATH)

	for filename in os.listdir(ORIGINAL_PATH):

		with open(os.path.join(ORIGINAL_PATH, filename), 'r') as f:
			original = f.read()

		soup = BeautifulSoup(original, 'lxml')
		content = soup.title.string.strip() + " " + soup.find('text').text.strip()

		if int(filename[-4:]) in random_samples:
			print("----------------------------------")
			print("Filename: ", filename)
			print("----------------------------------")
			print("Before: ")
			print(original)
			print("----------------------------------")
			print("After: ")
			print(content)
			print("----------------------------------")

		with open(os.path.join(ALTERED_PATH, filename), 'w') as fa:
			fa.write(content)

	print("Finished Processing")

In [18]:
def clear_alter():
	for filename in os.listdir(ALTERED_PATH):
		os.remove(os.path.join(ALTERED_PATH, filename))


In [20]:
# clear_alter()
create_alter()

----------------------------------
Filename:  cranfield0130
----------------------------------
Before: 
<DOC>
<DOCNO>
130
</DOCNO>
<TITLE>
the behaviour of non-linear systems .
</TITLE>
<AUTHOR>
clauser,f.h.
</AUTHOR>
<BIBLIO>
j.aero. scs. 23, 1958, 411.
</BIBLIO>
<TEXT>
  many of the phenomena that occur in the world around us are
governed by nonlinear relationships .  in the development of the
mathematical sciences, the difficulties of nonlinear analysis have
hindered the formulation of nonlinear concepts that would permit
us to understand such phenomena .  in the present article,
our progress in understanding the behavior of nonlinear systems
is reviewed and an attempt is made to present the resulting concepts
in such a way that they may be applied with some generality
to other problems .
</TEXT>
</DOC>

----------------------------------
After: 
the behaviour of non-linear systems . many of the phenomena that occur in the world around us are
governed by nonlinear relationships .  i

In [ ]:
def preprocess(seed=1):
	random.seed(seed)
	random_samples = random.sample(range(1, 1401), 5)

	for filename in os.listdir(ALTERED_PATH):
		with open(os.path.join(ALTERED_PATH, filename), 'r') as f:
			original = f.read()

		content = original.lower()


		if int(filename[-4:]) in random_samples:
			print("----------------------------------")
			print("LOWERCASE")
			print("----------------------------------")
			print("Filename: ", filename)
			print("----------------------------------")
			print("Before: ")
			print(original)
			print("----------------------------------")
			print("After: ")
			print(content)
			print("----------------------------------")

		content = word_tokenize(content)


		if int(filename[-4:]) in random_samples:
			print("----------------------------------")
			print("TOKENIZE")
			print("After: ")
			print(content)
			print("----------------------------------")

		content = [w for w in content if not w in stopwords.words('english')]


		if int(filename[-4:]) in random_samples:
			print("----------------------------------")
			print("STOPWORDS")
			print("After: ")
			print(content)
			print("----------------------------------")

		content = [w for w in content if not w in punctuation]


		if int(filename[-4:]) in random_samples:
			print("----------------------------------")
			print("PUNCTUATION")
			print("After: ")
			print(content)
			print("----------------------------------")

		content = [w for w in content if w.strip()]


		if int(filename[-4:]) in random_samples:
			print("----------------------------------")
			print("BLANKSPACE")
			print("After: ")
			print(content)
			print("----------------------------------")

		content = " ".join(content)

		with open(os.path.join(ALTERED_PATH, filename), 'w') as fa:
			fa.write(content)

	print("Finished Processing")
